In [ ]:
import torch
print(torch.version.cuda)  # '11.8'이 출력되어야 합니다.
print(torch.cuda.is_available())  # True가 출력되면 GPU가 정상적으로 인식된 것입니다.


In [1]:
import fitz
import json
import re
import nltk

PDF_FILE_PATH = "./data/GTM-Construction-Safety-Manual.pdf"
JSON_FILE_PATH = "./data/original_manual.json"

In [2]:
doc = fitz.open(PDF_FILE_PATH)
big_point = 0
middle_point = 0
small_point = 0
save_js = {}
for i, page in enumerate(doc):
    if i < 15 or i > 246:
        continue
    print(f"page {i+1}")
    blocks = page.get_text("dict")["blocks"][6:]
    for block in blocks:
        if "lines" not in block.keys():
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                text = span["text"]
                # Bold 체 중에 선택
                if "Bold" in span["font"]:        
                    # 대제목 가져오기
                    if re.match(r'^\d+\.\s', text.strip()):
                        print("BIG TITLE:", text)
                        big_point += 1
                        save_js[big_point] = {"title_text":text, "text":""}
                        middle_point = 0
                        small_point = 0
                        continue
                    # 중제목 가져오기
                    elif re.match(r'^\d+\.\d+\.\s', text.strip()):
                        print("MIDDLE TITLE:", text)
                        middle_point += 1
                        if "middle" not in save_js[big_point].keys():
                            save_js[big_point]["middle"] = {}
                        save_js[big_point]["middle"][middle_point] = {"title_text":text, "text":""}
                        small_point = 0
                        continue
                    # 소제목 가져오기
                    elif text != " " and text[0] == " " and text[1] != " ":
                        print("SMALL TITLE:", text)
                        small_point += 1
                        if "small" not in save_js[big_point]["middle"][middle_point].keys():
                            save_js[big_point]["middle"][middle_point]["small"] = {}
                        save_js[big_point]["middle"][middle_point]["small"][small_point] = {"title_text":text, "text":""}
                        continue
                # 각 제목의 text
                if middle_point == 0 and small_point == 0:
                    save_js[big_point]["text"] += text
                elif small_point == 0:
                    save_js[big_point]["middle"][middle_point]["text"] += text
                else:
                    save_js[big_point]["middle"][middle_point]["small"][small_point]["text"] += text
                print(big_point, middle_point,  small_point)

page 16
BIG TITLE: 1. Leadership and Governance 
MIDDLE TITLE: 1.1. Supplier Management System 
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
1 1 0
MIDDLE TITLE: 1.2. Leadership in Action 
1 2 0
1 2 0
1 2 0
1 2 0
1 2 0
MIDDLE TITLE: 1.3. Stop Work, Imminent Hazard, and Right to Refuse Unsafe Work 
1 3 0
1 3 0
1 3 0
1 3 0
page 17
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
1 3 0
SMALL TITLE:  Responsibilities 
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
page 18
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3 1
1 3

In [3]:
# 올바르게 나눠 졌는지 확인
for i in save_js.keys():
    if "middle" not in save_js[i].keys():
        print(f"{i}.")
        continue
    for j in save_js[i]["middle"].keys():
        if "small" not in save_js[i]["middle"][j].keys():
            print(f"{i}.{j}.")
            continue
        for k in save_js[i]["middle"][j]["small"].keys():
            print(f"{i}.{j}.{k}.")

1.1.
1.2.
1.3.1.
1.3.2.
2.1.
2.2.
2.3.
2.4.
2.5.
2.6.
3.1.
3.2.
3.3.
3.4.
3.5.
3.6.
3.7.
3.8.
3.9.
3.10.
3.11.
3.12.
3.13.
4.1.1.
4.2.1.
4.2.2.
4.3.1.
4.3.2.
4.3.3.
4.4.1.
4.5.1.
5.1.
5.2.
6.1.1.
6.2.1.
6.3.1.
6.4.1.
6.4.2.
6.4.3.
6.4.4.
6.4.5.
6.4.6.
6.4.7.
6.5.1.
6.6.1.
6.7.1.
6.7.2.
6.7.3.
6.8.1.
6.8.2.
6.8.3.
6.8.4.
6.8.5.
6.9.1.
6.9.2.
6.9.3.
6.9.4.
6.9.5.
6.9.6.
6.9.7.
6.9.8.
6.9.9.
6.9.10.
6.9.11.
6.9.12.
6.9.13.
6.9.14.
6.9.15.
6.9.16.
6.9.17.
6.9.18.
6.9.19.
6.9.20.
6.9.21.
6.10.1.
6.10.2.
6.10.3.
6.10.4.
6.10.5.
6.10.6.
6.10.7.
6.10.8.
6.10.9.
6.11.1.
6.11.2.
6.11.3.
6.12.1.
6.13.1.
6.14.1.
6.15.1.
6.16.1.
6.17.1.
6.18.1.
6.19.1.
6.19.2.
6.19.3.
6.19.4.
6.19.5.
6.19.6.
6.19.7.
6.19.8.
6.19.9.
6.19.10.
6.19.11.
6.19.12.
6.19.13.
6.19.14.
6.19.15.
6.19.16.
6.19.17.
6.19.18.
6.19.19.
6.19.20.
6.19.21.
6.19.22.
6.19.23.
6.20.1.
6.20.2.
6.20.3.
6.21.1.
6.21.2.
6.21.3.
6.21.4.
6.21.5.
6.21.6.
6.21.7.
6.22.1.
6.23.1.
6.23.2.
6.23.3.
6.23.4.
6.23.5.
6.24.1.
6.24.2.
6.24.3.
6.25.1.
6.

In [4]:
# 중제목 단위로 나눠서 json 파일로 저장
for i in save_js.keys():
    big_text = save_js[i]["title_text"].strip() + " "
    for j in save_js[i]["middle"].keys():
        middle_text = save_js[i]["middle"][j]["title_text"].strip() + " " + save_js[i]["text"] + save_js[i]["middle"][j]["text"]
        if "small"  in save_js[i]["middle"][j].keys(): 
            for k in save_js[i]["middle"][j]["small"].keys():
                middle_text +=  save_js[i]["middle"][j]["small"][k]["title_text"].strip() + " " + save_js[i]["middle"][j]["small"][k]["text"]
        
        # 딕셔너리를 JSON 파일로 저장
        with open(f'./text_to_json/data{i}-{j}.json', 'w') as json_file:
            json.dump(big_text+middle_text, json_file, indent=4)